In [1]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]
!apt-get install swig
!pip3 install box2d box2d-kengz

  Using cached box2d-py-2.3.8.tar.gz (374 kB)
  ERROR: Command errored out with exit status 1:
   command: /Users/jeanette/opt/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/dp/09wb07vs11g9p_5h1vssmq140000gn/T/pip-install-ln5cw244/box2d-py_42ed324a3c0d4ae2a5a7773f5ff4437a/setup.py'"'"'; __file__='"'"'/private/var/folders/dp/09wb07vs11g9p_5h1vssmq140000gn/T/pip-install-ln5cw244/box2d-py_42ed324a3c0d4ae2a5a7773f5ff4437a/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/dp/09wb07vs11g9p_5h1vssmq140000gn/T/pip-wheel-2_h6sd0e
       cwd: /private/var/folders/dp/09wb07vs11g9p_5h1vssmq140000gn/T/pip-install-ln5cw244/box2d-py_42ed324a3c0d4ae2a5a7773f5ff4437a/
  Complete output (17 lines):
  Using setuptools (version 52.0.0.post20210125).
  running bdist_wheel
  running build
  runn

In [28]:
import gym
import numpy as np
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import copy
import csv
from collections import deque

In [29]:
NOISE = 0.1

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [31]:
class Actor(nn.Module):
    
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.seq = nn.Sequential(
                nn.Linear(state_dim, 400),
                nn.ReLU(),
                nn.Linear(400, 300),
                nn.ReLU(),
                nn.Linear(300, action_dim),
                nn.Tanh()
                )
        self.max_action = max_action

    def forward(self, x):
        return self.seq(x) * self.max_action

In [32]:
class Critic(nn.Module):
 
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.seq = nn.Sequential(
                nn.Linear(state_dim + action_dim, 400),
                nn.ReLU(),
                nn.Linear(400, 300),
                nn.ReLU(),
                nn.Linear(300, 1),
                )

    def forward(self, x, u):
        xu = torch.cat([x, u], 1)
        return self.seq(xu)

In [33]:
class ReplayBuffer(object):
    def __init__(self):
        self.storage = []
        self.ptr = 0

    def add(self, data):
        max_size = 1e6
        if len(self.storage) == max_size:
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % max_size
        else:
            self.storage.append(data)
    
    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        s, s1, a, r, d = [], [], [], [], []

        for i in ind: 
            S, S1, A, R, D = self.storage[i]
            s.append(np.array(S, copy=False))
            s1.append(np.array(S1, copy=False))
            a.append(np.array(A, copy=False))
            r.append(np.array(R, copy=False))
            d.append(np.array(D, copy=False))

        return np.array(s), np.array(s1), np.array(a), np.array(r).reshape(-1, 1), np.array(d).reshape(-1, 1)   

In [54]:
class TD3(object): 
    def __init__(self, state_dim, action_dim, max_action):
        self.ac = Actor(state_dim, action_dim, max_action).to(device)
        self.ac_target = Actor(state_dim, action_dim, max_action).to(device)
        self.ac_target.load_state_dict(self.ac.state_dict())
        self.ac_optimizer = torch.optim.Adam(self.ac.parameters())

        self.cr1 = Critic(state_dim, action_dim).to(device)
        self.cr1_target = Critic(state_dim, action_dim).to(device)
        self.cr1_target.load_state_dict(self.cr1.state_dict())
        self.cr1_optimizer = torch.optim.Adam(self.cr1.parameters())
        
        self.cr2 = Critic(state_dim, action_dim).to(device)
        self.cr2_target = Critic(state_dim, action_dim).to(device)
        self.cr2_target.load_state_dict(self.cr2.state_dict())
        self.cr2_optimizer = torch.optim.Adam(self.cr2.parameters())

        self.max_action = max_action

    def select_action(self, state):
        return self.ac(self.tensor(state.reshape(1, -1))).cpu().data.numpy().flatten()
    
    def tensor(self, val):
        return torch.FloatTensor(val).to(device)

    def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, \
              tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):
        
        for it in range(iterations):

            s, s1, a, r, d = replay_buffer.sample(batch_size)
            state, next_state, action, reward, done = self.tensor(s), self.tensor(s1), self.tensor(a), self.tensor(r), self.tensor(1 - d)

            # Select action according to policy and add clipped noise 
            noise = (
                    torch.FloatTensor(a).data.normal_(0, NOISE).to(device)
            ).clamp(-noise_clip, noise_clip)
            next_action = (self.ac_target(next_state) + noise).clamp(-self.max_action, self.max_action)

            # Compute the target Q value
            target_Q1, target_Q2 = self.cr1_target(next_state, next_action), self.cr2_target(next_state, next_action)
            target_Q = reward + (done * discount * torch.min(target_Q1, target_Q2)).detach()

            # Get current Q estimates
            current_Q1, current_Q2 = self.cr1(state, action), self.cr2(state, action)

            # Compute critic loss
            cr1_loss, cr2_loss = F.mse_loss(current_Q1, target_Q), F.mse_loss(current_Q2, target_Q) 

            # Optimize the critic
            self.cr1_optimizer.zero_grad()
            cr1_loss.backward()
            self.cr1_optimizer.step()
            
            self.cr2_optimizer.zero_grad()
            cr2_loss.backward()
            self.cr2_optimizer.step()

            # Delayed policy updates
            if it % policy_freq == 0:

                # Compute actor loss
                ac_loss = -self.cr1.forward(state, self.ac(state)).mean()
                # Optimize the actor 
                self.ac_optimizer.zero_grad()
                ac_loss.backward()
                self.ac_optimizer.step()

                # Update the frozen target models
                self.update_param(tau)

    def update_param(self, tau):
        if tau == None:
            return
        
        cr1_param, cr1_param_target = self.cr1.parameters(), self.cr1_target.parameters()
        while True:
            try:
                elem1 = next(cr1_param)
                elem2 = next(cr1_param_target)
            except StopIteration:
                break
            elem2.data.copy_(tau * elem1.data + (1 - tau) * elem2.data)

        cr2_param, cr2_param_target = self.cr2.parameters(), self.cr2_target.parameters()
        while True:
            try:
                elem3 = next(cr2_param)
                elem4 = next(cr2_param_target)
            except StopIteration:
                break
            elem4.data.copy_(tau * elem3.data + (1 - tau) * elem4.data)

        ac_param, ac_param_target = self.ac.parameters(), self.ac_target.parameters()
        while True:
            try:
                elem5 = next(ac_param)
                elem6 = next(ac_param_target)
            except StopIteration:
                break
            elem6.data.copy_(tau * elem5.data + (1 - tau) * elem6.data)

                    

In [55]:
if __name__ == '__main__':

    # Twin Delayed Deep Deterministic (TD3) policy gradient algorithm
    def twin_ddd_train(n_episodes=2000, save_every=10):
        file_log = open('TD3_reward.csv',"w+")
        file_log.write('episode,step,reward\n')
        scores_deque = deque(maxlen=100)
        scores_array = []
        avg_scores_array = []    

        replay_buf = ReplayBuffer()                 # Init ReplayBuffer

        total_timesteps = 0

        for i_episode in range(1, n_episodes+1):

            timestep = 0
            total_reward = 0

            # Reset environment
            state, done = env.reset(), False

            while True:

                # Select action randomly or according to policy
                if total_timesteps < start_timestep:
                    action = env.action_space.sample()
                else:
                    action = agent.select_action(np.array(state))
                    if std_noise != 0: 
                        shift_action = np.random.normal(0, std_noise, size=action_dim)
                        action = (action + shift_action).clip(env.action_space.low, env.action_space.high)

                # Perform action
                new_state, reward, done, _ = env.step(action) 
                done_bool = 0 if timestep + 1 == env._max_episode_steps else float(done)
                total_reward += reward                          # full episode reward

                # Store every timestep in replay buffer
                replay_buf.add((state, new_state, action, reward, done_bool))
                state = new_state

                timestep += 1     
                total_timesteps += 1

                if done:
                    break

            scores_deque.append(total_reward)
            scores_array.append(total_reward)

            avg_score = np.mean(scores_deque)
            avg_scores_array.append(avg_score)

            # train_by_episode(time_start, i_episode)
            to_print = '{},{},{}\n'.format(i_episode, total_timesteps, avg_score)
            print('Ep. {}, Timestep {},  Ep.Timesteps {}, Score: {:.2f}, Avg.Score: {:.2f} '\
                    .format(i_episode, total_timesteps, timestep, \
                            total_reward, avg_score))
            file_log.write(to_print)
            file_log.flush()
            agent.train(replay_buf, timestep)

        return scores_array, avg_scores_array

    
    start_timestep=1e4

    std_noise=0.1

    env = gym.make('BipedalWalker-v3')
    state = env.reset()
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0] 
    max_action = float(env.action_space.high[0])
    agent = TD3(state_dim, action_dim, max_action)
    scores, avg_scores = twin_ddd_train()

Ep. 1, Timestep 62,  Ep.Timesteps 62, Score: -100.28, Avg.Score: -100.28 
Ep. 2, Timestep 134,  Ep.Timesteps 72, Score: -100.48, Avg.Score: -100.38 
Ep. 3, Timestep 191,  Ep.Timesteps 57, Score: -107.68, Avg.Score: -102.81 
Ep. 4, Timestep 1791,  Ep.Timesteps 1600, Score: -76.53, Avg.Score: -96.24 
Ep. 5, Timestep 1884,  Ep.Timesteps 93, Score: -104.09, Avg.Score: -97.81 
Ep. 6, Timestep 3484,  Ep.Timesteps 1600, Score: -82.56, Avg.Score: -95.27 
Ep. 7, Timestep 3606,  Ep.Timesteps 122, Score: -122.47, Avg.Score: -99.15 
Ep. 8, Timestep 3663,  Ep.Timesteps 57, Score: -112.90, Avg.Score: -100.87 
Ep. 9, Timestep 3726,  Ep.Timesteps 63, Score: -101.52, Avg.Score: -100.94 
Ep. 10, Timestep 5326,  Ep.Timesteps 1600, Score: -76.92, Avg.Score: -98.54 
Ep. 11, Timestep 6926,  Ep.Timesteps 1600, Score: -101.05, Avg.Score: -98.77 
Ep. 12, Timestep 7001,  Ep.Timesteps 75, Score: -107.96, Avg.Score: -99.54 
Ep. 13, Timestep 7070,  Ep.Timesteps 69, Score: -117.96, Avg.Score: -100.95 
Ep. 14, Times

Ep. 107, Timestep 32023,  Ep.Timesteps 82, Score: -97.33, Avg.Score: -108.28 
Ep. 108, Timestep 32177,  Ep.Timesteps 154, Score: -132.43, Avg.Score: -108.48 
Ep. 109, Timestep 32229,  Ep.Timesteps 52, Score: -101.72, Avg.Score: -108.48 
Ep. 110, Timestep 32344,  Ep.Timesteps 115, Score: -95.88, Avg.Score: -108.67 
Ep. 111, Timestep 32436,  Ep.Timesteps 92, Score: -109.18, Avg.Score: -108.75 
Ep. 112, Timestep 32507,  Ep.Timesteps 71, Score: -97.77, Avg.Score: -108.65 
Ep. 113, Timestep 32596,  Ep.Timesteps 89, Score: -99.47, Avg.Score: -108.46 
Ep. 114, Timestep 32652,  Ep.Timesteps 56, Score: -101.40, Avg.Score: -108.47 
Ep. 115, Timestep 32729,  Ep.Timesteps 77, Score: -99.94, Avg.Score: -108.58 
Ep. 116, Timestep 32799,  Ep.Timesteps 70, Score: -100.20, Avg.Score: -108.58 
Ep. 117, Timestep 34399,  Ep.Timesteps 1600, Score: -99.38, Avg.Score: -108.56 
Ep. 118, Timestep 34738,  Ep.Timesteps 339, Score: -141.79, Avg.Score: -108.78 
Ep. 119, Timestep 36338,  Ep.Timesteps 1600, Score: -

Ep. 210, Timestep 130992,  Ep.Timesteps 1600, Score: -105.35, Avg.Score: -113.36 
Ep. 211, Timestep 132592,  Ep.Timesteps 1600, Score: -102.38, Avg.Score: -113.29 
Ep. 212, Timestep 132756,  Ep.Timesteps 164, Score: -139.39, Avg.Score: -113.70 
Ep. 213, Timestep 134356,  Ep.Timesteps 1600, Score: -70.39, Avg.Score: -113.41 
Ep. 214, Timestep 135956,  Ep.Timesteps 1600, Score: -96.22, Avg.Score: -113.36 
Ep. 215, Timestep 137556,  Ep.Timesteps 1600, Score: -136.08, Avg.Score: -113.72 
Ep. 216, Timestep 139156,  Ep.Timesteps 1600, Score: -82.48, Avg.Score: -113.55 
Ep. 217, Timestep 140756,  Ep.Timesteps 1600, Score: -131.84, Avg.Score: -113.87 
Ep. 218, Timestep 142356,  Ep.Timesteps 1600, Score: -164.68, Avg.Score: -114.10 
Ep. 219, Timestep 143956,  Ep.Timesteps 1600, Score: -87.87, Avg.Score: -113.92 
Ep. 220, Timestep 145556,  Ep.Timesteps 1600, Score: -70.05, Avg.Score: -113.42 
Ep. 221, Timestep 147156,  Ep.Timesteps 1600, Score: -94.21, Avg.Score: -112.84 
Ep. 222, Timestep 14875

Ep. 311, Timestep 254057,  Ep.Timesteps 1600, Score: -118.49, Avg.Score: -101.68 
Ep. 312, Timestep 254144,  Ep.Timesteps 87, Score: -125.57, Avg.Score: -101.54 
Ep. 313, Timestep 254217,  Ep.Timesteps 73, Score: -121.46, Avg.Score: -102.05 
Ep. 314, Timestep 254374,  Ep.Timesteps 157, Score: -110.08, Avg.Score: -102.19 
Ep. 315, Timestep 255974,  Ep.Timesteps 1600, Score: -52.50, Avg.Score: -101.35 
Ep. 316, Timestep 257574,  Ep.Timesteps 1600, Score: -65.16, Avg.Score: -101.18 
Ep. 317, Timestep 258921,  Ep.Timesteps 1347, Score: -184.80, Avg.Score: -101.71 
Ep. 318, Timestep 260521,  Ep.Timesteps 1600, Score: -121.12, Avg.Score: -101.27 
Ep. 319, Timestep 262121,  Ep.Timesteps 1600, Score: -92.42, Avg.Score: -101.32 
Ep. 320, Timestep 263721,  Ep.Timesteps 1600, Score: -111.47, Avg.Score: -101.73 
Ep. 321, Timestep 265321,  Ep.Timesteps 1600, Score: -107.15, Avg.Score: -101.86 
Ep. 322, Timestep 266921,  Ep.Timesteps 1600, Score: -96.04, Avg.Score: -101.77 
Ep. 323, Timestep 268521,

Ep. 412, Timestep 330323,  Ep.Timesteps 452, Score: -162.72, Avg.Score: -129.34 
Ep. 413, Timestep 330399,  Ep.Timesteps 76, Score: -119.74, Avg.Score: -129.33 
Ep. 414, Timestep 330545,  Ep.Timesteps 146, Score: -134.82, Avg.Score: -129.57 
Ep. 415, Timestep 332145,  Ep.Timesteps 1600, Score: -153.74, Avg.Score: -130.59 
Ep. 416, Timestep 332278,  Ep.Timesteps 133, Score: -124.42, Avg.Score: -131.18 
Ep. 417, Timestep 332487,  Ep.Timesteps 209, Score: -123.97, Avg.Score: -130.57 
Ep. 418, Timestep 332742,  Ep.Timesteps 255, Score: -149.45, Avg.Score: -130.85 
Ep. 419, Timestep 334342,  Ep.Timesteps 1600, Score: -165.25, Avg.Score: -131.58 
Ep. 420, Timestep 334512,  Ep.Timesteps 170, Score: -120.25, Avg.Score: -131.67 
Ep. 421, Timestep 334789,  Ep.Timesteps 277, Score: -135.88, Avg.Score: -131.96 
Ep. 422, Timestep 335554,  Ep.Timesteps 765, Score: -150.94, Avg.Score: -132.51 
Ep. 423, Timestep 335655,  Ep.Timesteps 101, Score: -103.29, Avg.Score: -132.61 
Ep. 424, Timestep 337255,  

...

Ep. 1127, Timestep 1308056,  Ep.Timesteps 928, Score: 277.03, Avg.Score: 260.34 
Ep. 1128, Timestep 1309128,  Ep.Timesteps 1072, Score: 263.38, Avg.Score: 260.36 
Ep. 1129, Timestep 1310041,  Ep.Timesteps 913, Score: 277.38, Avg.Score: 260.58 
Ep. 1130, Timestep 1311036,  Ep.Timesteps 995, Score: 268.70, Avg.Score: 260.69 
Ep. 1131, Timestep 1311959,  Ep.Timesteps 923, Score: 275.82, Avg.Score: 260.82 
Ep. 1132, Timestep 1312970,  Ep.Timesteps 1011, Score: 265.13, Avg.Score: 260.90 
Ep. 1133, Timestep 1313877,  Ep.Timesteps 907, Score: 275.05, Avg.Score: 260.99 
Ep. 1134, Timestep 1314808,  Ep.Timesteps 931, Score: 275.20, Avg.Score: 261.12 
Ep. 1135, Timestep 1315860,  Ep.Timesteps 1052, Score: 266.15, Avg.Score: 261.20 
Ep. 1136, Timestep 1316757,  Ep.Timesteps 897, Score: 277.84, Avg.Score: 261.42 
Ep. 1137, Timestep 1317686,  Ep.Timesteps 929, Score: 278.02, Avg.Score: 261.61 
Ep. 1138, Timestep 1318618,  Ep.Timesteps 932, Score: 274.92, Avg.Score: 261.68 
Ep. 1139, Timestep 131952

Ep. 1228, Timestep 1401856,  Ep.Timesteps 1029, Score: 266.52, Avg.Score: 268.65 
Ep. 1229, Timestep 1402815,  Ep.Timesteps 959, Score: 272.07, Avg.Score: 268.59 
Ep. 1230, Timestep 1403765,  Ep.Timesteps 950, Score: 274.48, Avg.Score: 268.65 
Ep. 1231, Timestep 1404704,  Ep.Timesteps 939, Score: 273.12, Avg.Score: 268.62 
Ep. 1232, Timestep 1405685,  Ep.Timesteps 981, Score: 269.85, Avg.Score: 268.67 
Ep. 1233, Timestep 1406616,  Ep.Timesteps 931, Score: 274.57, Avg.Score: 268.67 
Ep. 1234, Timestep 1407589,  Ep.Timesteps 973, Score: 275.09, Avg.Score: 268.67 
Ep. 1235, Timestep 1408527,  Ep.Timesteps 938, Score: 275.08, Avg.Score: 268.75 
Ep. 1236, Timestep 1409479,  Ep.Timesteps 952, Score: 271.93, Avg.Score: 268.70 
Ep. 1237, Timestep 1410427,  Ep.Timesteps 948, Score: 273.59, Avg.Score: 268.65 
Ep. 1238, Timestep 1411426,  Ep.Timesteps 999, Score: 268.13, Avg.Score: 268.58 
Ep. 1239, Timestep 1412371,  Ep.Timesteps 945, Score: 273.03, Avg.Score: 268.53 
Ep. 1240, Timestep 1413294,

Ep. 1330, Timestep 1497541,  Ep.Timesteps 924, Score: 275.60, Avg.Score: 274.95 
Ep. 1331, Timestep 1498513,  Ep.Timesteps 972, Score: 271.36, Avg.Score: 274.93 
Ep. 1332, Timestep 1499467,  Ep.Timesteps 954, Score: 273.50, Avg.Score: 274.97 
Ep. 1333, Timestep 1500360,  Ep.Timesteps 893, Score: 277.53, Avg.Score: 275.00 
Ep. 1334, Timestep 1501245,  Ep.Timesteps 885, Score: 278.73, Avg.Score: 275.04 
Ep. 1335, Timestep 1502142,  Ep.Timesteps 897, Score: 278.96, Avg.Score: 275.07 
Ep. 1336, Timestep 1503051,  Ep.Timesteps 909, Score: 277.54, Avg.Score: 275.13 
Ep. 1337, Timestep 1503989,  Ep.Timesteps 938, Score: 271.86, Avg.Score: 275.11 
Ep. 1338, Timestep 1504860,  Ep.Timesteps 871, Score: 281.05, Avg.Score: 275.24 
Ep. 1339, Timestep 1505759,  Ep.Timesteps 899, Score: 278.27, Avg.Score: 275.29 
Ep. 1340, Timestep 1506667,  Ep.Timesteps 908, Score: 276.56, Avg.Score: 275.33 
Ep. 1341, Timestep 1507577,  Ep.Timesteps 910, Score: 276.61, Avg.Score: 275.38 
Ep. 1342, Timestep 1508484, 

Ep. 1432, Timestep 1591681,  Ep.Timesteps 933, Score: 282.44, Avg.Score: 277.75 
Ep. 1433, Timestep 1592599,  Ep.Timesteps 918, Score: 280.79, Avg.Score: 277.78 
Ep. 1434, Timestep 1593504,  Ep.Timesteps 905, Score: 276.80, Avg.Score: 277.76 
Ep. 1435, Timestep 1594432,  Ep.Timesteps 928, Score: 277.35, Avg.Score: 277.74 
Ep. 1436, Timestep 1595335,  Ep.Timesteps 903, Score: 279.96, Avg.Score: 277.77 
Ep. 1437, Timestep 1596219,  Ep.Timesteps 884, Score: 280.85, Avg.Score: 277.86 
Ep. 1438, Timestep 1597086,  Ep.Timesteps 867, Score: 283.11, Avg.Score: 277.88 
Ep. 1439, Timestep 1597999,  Ep.Timesteps 913, Score: 280.96, Avg.Score: 277.90 
Ep. 1440, Timestep 1598929,  Ep.Timesteps 930, Score: 281.40, Avg.Score: 277.95 
Ep. 1441, Timestep 1599877,  Ep.Timesteps 948, Score: 276.29, Avg.Score: 277.95 
Ep. 1442, Timestep 1600910,  Ep.Timesteps 1033, Score: 269.18, Avg.Score: 277.84 
Ep. 1443, Timestep 1601792,  Ep.Timesteps 882, Score: 282.92, Avg.Score: 277.94 
Ep. 1444, Timestep 1602702,

KeyboardInterrupt: 